# Design

# GNNS Modules

## ATGM332D-5N31

### Properties
**GNSS**:
GPS, BDS: l1, 1575,42 MHz; b1, 1561,098 MHz

**Communication**
- TXD1, RXD1: Communication happens through UART according to the NMEA0183-protocol.
- SDA, SCL: I2C data interface
- TXD2, RXD2: auxiliary serial data output.\
  
-> Route all these to relevant output connectors on the testboard

**Input power: VBAT (RTC / SRAM supply), VCC (power input)**: 3.3 V \
-> Supply from external pin and put LDO in between

**Output power (VCC_RF)**: 3.3 V \
-> Connect to external LNA (use passive antenna)

**nReset**: Floating when not in use, pull-down when reset \
-> Connect to power supply with button (high by default high)

**RFin: antenna signal input** \
-> Connect to LNA, assure yourself of correct impedance matching

**PPS**: +- 100 ms pulse every second of which the rising edge is aligned with UTC time. \
-> Connect to external pin
### Electrical parameters
**Logic Input**: high / low: (.7/.2)Vcc \
**Logic Output**: high / low: (Vcc-0.5, .4Vcc) \
**Antenna gain (input)**: 15-30 dB

### Module diagram
<img src="../images/wireless/gps_ATGM332D_module_diagram.png" width=600 />

### Advised passive circuitry
- AT2659 LNA
- 6.9 nH, 470 pF output impedance matching.
- 0.1 uF decoupling capacitor

<img src="../images/wireless/gps_ATGM332D_passive_schematic.png" width=800 />

# LNA

## AT2659
### Properties
**Input P1DB**: -14 dBm \
**Input IP3**: -5 dBm \
**Power gain**: 20 dB \
**Working frequency**: 1550-1615 MHz \
**Current consumption**: 4.3 mA \
**Impedance Matching**: internally 50 ohms \
**Noise**: 1 dB max 
### Peripheral components
* Bypass capacitor, input matching and DC-block capacitor \
-> Make sure to check the HF characteristics for these components

Signal strength after passing through antenna: 
* $P_{gain} = 10×log (P_{out} / P_{in})$ \
Example:
* If an antenna gain is 2 dBi
* The amplifier gain is 20 dB, which means the total gain becomes $20 + 2 = 22$
* Input power should be multiplied with amplification power: $P_{input}*P_{amp}$ ->  $20*log(P_{input}*P_{amp}) = 20*log(P_{input}) + 20*log(P_{amp})$

# RF Switch

## PE4259

### Properties
**Frequency Range**: 3 GHz \
**Insertion loss**: 0.35 dB \
**Isolation**: 30 dB \


### Peripheral components
* No DC blocking capacitor, use 2x0.5pF capacitor at RFC input \

<img src="../images/wireless/gps_pe4259.png" width=400 />

**Sources:**
> TI Antenna selection guide: https://www.ti.com/lit/an/swra161b/swra161b.pdf?ts=17241%2017291346 \
> RF-switch data: https://wmsc.lcsc.com/wmsc/upload/file/pdf/v2/lcsc/1912111437_Skyworks-Solutions-SKY13489-001_C429943.pdf 

# Example

## NEO-M8P

### Frequencies
* GLONASS: 1602 MHz + k*562.5 (k = -7..6), satellite frequency number
* BeiDou: 1561.098 MHz, B11 signal broadcast
* Galileo: 1559 - 1610 MHz, GPS L1 frequency band (E1-B/C-signals)
* GPS: 1575.42 MHz L1C/A signals
### Clocks
* Oscillator
* RTC (driven by 32 kHz clock)
### Antenna
Designed with passive and active antennas.
* 15 dB minimum gain
* 50 / 30 dB maximum gain
* 1.5 dB maximum noise figure

LNA_EN-pin allows to switch off power to the active antenna

## Design recommendations
### RF interference
Use 2 resistors / ferrite beads (> 33 ohms) as closely as possible to the I/O input of the GNSS antenna for shielding.

## Passive antenna design

<img src="../images/wireless/gps_TFGPS_ublox_hw_integration.png" width=600 />

## Bandwidth Filter (SAW DBP.1575.W.A.30)
### Characteristics
<img src="../images/wireless/gps_TFGPS_SAW_characteristics.png" width=400 />

#### Impedance

Impedance matched to 50 ohms

## MAX2655


## Antenna
### Characteristics
<img src="../images/wireless/gps_TFGPS_antenna_characteristics.png" width=400 />

#### Impedance
50 ohms.

## Schematic design
<img src="../images/wireless/gps_TFGPS_rf_schematic.png" width=600 />

### Filtering on RF_POWER line
#### Filtering on RF line
Possibility for filtering on RF line present, for now however no ferrite placed.

#### C11
DC-blocking capacitor between filter and amplifier.
- Probably to make sure that the RF_IN does not break due to amplification of DC offset from amplifier.

#### C14
DC blocking capacitor preventing DC signals from making it to the amplifier.

#### RF5
LP-filter
Cutoff frequency: $1/(2*\pi*sqrt(5.6e-9+1.5e-12)) = 1736 MHz$, additional filtering of antenna input.
Could also be external matching to RFIN. (referenced in "typical operating circuit".

#### BIAS
Sets the IP3 (third order intercept) and supply current.
Here chosen to be 390 ohms.

#### RFOUT
DC-bias on the pin selects gain mode. The gain is attenutated by 13 dB.
Check tables 1 and 2 of the datasheet for relevant information

**Sources**:

> Datasheet: https://content.u-blox.com/sites/default/files/NEO-M8-FW3_DataSheet_UBX-15031086.pdf \
> Hardware integration manual: https://content.u-blox.com/sites/default/files/NEO-8Q-NEO-M8-FW3_HIM_UBX-15029985.pdf \
GPS compendium: https://content.u-blox.com/sites/default/files/products/documents/GPS-Compendium_Book_%28GPS-X-02007%29.pdf \
> Bandwidth filter: https://nl.mouser.com/datasheet/2/398/DBP_1575_W_A_30-3400304.pdf  \
> Third-order intercept: https://www.youtube.com/watch?v=m-2H8ddSwTI \
> TI LC filter design: https://www.ti.com/lit/an/slaa701a/slaa701a.pdf?ts=1724088213231&ref_url=https%253A%252F%252Fwww.google.com%252F \